In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

2024-10-21 11:28:17.610229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the training data
train_df = pd.read_csv('TRAIN_Reco_2021_2022_2023.csv')

# Display the first few rows
train_df.head()

,ExecutionTime,ID,high,low,close,volume
0,2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
1,2021-01-06 22:00:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2,2021-01-06 22:15:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
3,2021-01-06 22:30:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
4,2021-01-06 22:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0


In [3]:
train_df.shape


(69805344, 6)

In [4]:
import pandas as pd

# Step 1: Check for rows where 'volume' is 0.0 and remove those rows
train_df_cleaned = train_df[train_df['volume'] != 0.0]

# Step 2: Print the shape of the cleaned dataframe
print("Shape after removing rows with 'volume' equal to 0.0:", train_df_cleaned.shape)


Shape after removing rows with 'volume' equal to 0.0: (4518103, 6)


In [5]:
# Get basic statistics about the dataset
train_df_cleaned.describe()

,high,low,close,volume
count,4.518103e+06,4.518103e+06,4.518103e+06,4.518103e+06
mean,1.544531e+02,1.481643e+02,1.512916e+02,7.615150e+00
std,1.230588e+02,1.194735e+02,1.211441e+02,1.472579e+01
min,-2.318860e+03,-2.882710e+03,-2.318860e+03,2.500000e-02
25%,7.796000e+01,7.410000e+01,7.600000e+01,3.750000e-01
50%,1.176200e+02,1.127900e+02,1.151300e+02,1.900000e+00
75%,1.978400e+02,1.910600e+02,1.944500e+02,7.775000e+00
max,9.999000e+03,7.655300e+03,9.000000e+03,1.201825e+03


In [14]:
# Display the first few rows
train_df_cleaned.head()

,ExecutionTime,ID,high,low,close,volume
90,2021-01-07 20:15:00+01:00,Fri00Q1,57.28,55.20,57.28,1.125
91,2021-01-07 20:30:00+01:00,Fri00Q1,61.92,59.70,61.30,5.150
93,2021-01-07 21:00:00+01:00,Fri00Q1,59.95,56.09,56.09,0.200
94,2021-01-07 21:15:00+01:00,Fri00Q1,63.75,60.99,63.47,2.525
95,2021-01-07 21:30:00+01:00,Fri00Q1,63.50,62.60,63.24,2.425


In [15]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Define the forecasting horizon and look-back window
n_timesteps = 10  # Look-back window
forecast_horizon = 10  # Number of steps to predict

# Initialize the scaler
scaler = MinMaxScaler()

# Loop through each asset and scale the features, then prepare sliding windows
asset_ids = train_df_cleaned['ID'].unique()
X, y = [], []

for asset in asset_ids:
    # Extract the data for this asset
    asset_data = train_df_cleaned[train_df_cleaned['ID'] == asset][['high', 'low', 'close', 'volume']].values
    
    # Scale the data using MinMaxScaler
    asset_data_scaled = scaler.fit_transform(asset_data)  # Scaling for each asset separately

    # Create sliding windows for the asset
    for i in range(len(asset_data_scaled) - n_timesteps - forecast_horizon):
        X.append(asset_data_scaled[i:i+n_timesteps])  # Past `n_timesteps` for input
        y.append(asset_data_scaled[i+n_timesteps:i+n_timesteps+forecast_horizon])  # Next 10 timesteps for output

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Check the shapes
print(f"Input shape: {X.shape}")  # Expected shape: (samples, timesteps, features)
print(f"Output shape: {y.shape}")  # Expected shape: (samples, forecast_horizon, features)


Input shape: (4504663, 10, 4)
Output shape: (4504663, 10, 4)


In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

# Step 1: Define the LSTM model
model = Sequential()

# Add LSTM layer
model.add(LSTM(units=100, return_sequences=False, input_shape=(n_timesteps, 4)))  # 4 features: HLCV

# Add Dense layer to output the predictions for the next 10 timesteps (4 features per timestep)
model.add(Dense(forecast_horizon * 4))  # 4 features per timestep

# Step 2: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 3: Train the model
history = model.fit(X, y.reshape(y.shape[0], -1), epochs=10, batch_size=32, validation_split=0.2)

# The shape of y is reshaped to (samples, forecast_horizon * 4) for each prediction


/Users/saarthakkataria/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 494s 4ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 2/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 484s 4ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 3/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 3074s 27ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 4/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 514s 5ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 5/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 502s 4ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 6/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 510s 5ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 7/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 471s 4ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 8/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 484s 4ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 9/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 493s 4ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 10/10
112617/112617 ━━━━━━━━━━━━━━━━━━━━ 507s 4ms/step - loss: 0.0025 - val_loss: 0.0024


In [17]:
# Print model summary to see the architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100)            │        42,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │         4,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,122 (539.54 KB)

 Trainable params: 46,040 (179.84 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 92,082 (359.70 KB)

In [18]:
import pandas as pd

# Load the test data
test_df = pd.read_csv('TEST_Reco_2024.csv')

# Apply the scaler (fitted on training data) to scale the test data
test_features = test_df[['high', 'low', 'close', 'volume']].values
test_scaled = scaler.transform(test_features)

# Create sliding windows for the test data
X_test = []
for i in range(len(test_scaled) - n_timesteps - forecast_horizon):
    X_test.append(test_scaled[i:i+n_timesteps])

# Convert to numpy array
X_test = np.array(X_test)

# Check the shape of X_test
print(f"Shape of X_test: {X_test.shape}")


Shape of X_test: (14648236, 10, 4)


In [19]:
# Step 4: Make predictions on the test data
predictions = model.predict(X_test)

# Reshape the predictions to the correct format
predictions = predictions.reshape(predictions.shape[0], forecast_horizon, 4)  # 4 features: HLCV

# Step 5: Inverse transform the predictions back to the original scale
predictions_original = np.array([scaler.inverse_transform(pred) for pred in predictions])

# Step 6: Extract the actual HLCV values from the test data (assuming the same look-back period)
y_test = test_df[['high', 'low', 'close', 'volume']].values[n_timesteps:n_timesteps+len(predictions)]


457758/457758 ━━━━━━━━━━━━━━━━━━━━ 875s 2ms/step


In [32]:
# Check the total size of the test data for high, low, close, volume
print(f"Total number of rows in the test data: {test_df[['high', 'low', 'close', 'volume']].shape[0]}")


Total number of rows in the test data: 14648256


In [36]:
# Check total available test data size vs predictions size
print(f"Total rows in test data: {len(test_df)}")
print(f"Required rows based on predictions: {len(predictions_original) * forecast_horizon}")

# Ensure slicing of y_test to fit exactly the available data
available_size = min(len(test_df) - n_timesteps, len(predictions_original) * forecast_horizon)

# Adjust y_test to only slice the available number of rows
y_test = test_df[['high', 'low', 'close', 'volume']].values[n_timesteps:n_timesteps + available_size]

print(f"Size of y_test after slicing: {y_test.size}")

# Try reshaping with the adjusted size
if y_test.size == len(predictions_original) * forecast_horizon * 4:
    y_test_reshaped = y_test.reshape(predictions_original.shape[0], forecast_horizon, 4)
    print(f"Reshaped y_test: {y_test_reshaped.shape}")
else:
    print(f"y_test size {y_test.size} does not match expected {len(predictions_original) * forecast_horizon * 4}")


Total rows in test data: 14648256
Required rows based on predictions: 146482360
Size of y_test after slicing: 58592984
y_test size 58592984 does not match expected 585929440


In [37]:
# Adjust the number of predictions to fit the available test data
available_predictions = min(len(predictions_original), len(y_test) // forecast_horizon)

# Slice the predictions and y_test to match
predictions_original = predictions_original[:available_predictions]
y_test = y_test[:available_predictions * forecast_horizon]

# Reshape y_test to match the adjusted predictions
y_test_reshaped = y_test.reshape(predictions_original.shape[0], forecast_horizon, 4)

# Check the sizes
print(f"Shape of y_test_reshaped: {y_test_reshaped.shape}")
print(f"Shape of predictions_original: {predictions_original.shape}")


Shape of y_test_reshaped: (1464824, 10, 4)
Shape of predictions_original: (1464824, 10, 4)


In [28]:
print(f"Length of predictions_original: {len(predictions_original)}")
print(f"Forecast horizon: {forecast_horizon}")
print(f"Required size for y_test: {required_size}")
print(f"Shape of y_test before reshaping: {y_test.shape}")


Length of predictions_original: 14648236
Forecast horizon: 10
Required size for y_test: 146482360
Shape of y_test before reshaping: (14648246, 4)


In [38]:
# Define the sMAPE function
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    return np.mean(diff) * 100

# Step 8: Calculate sMAPE between the predictions and actual values
smape_value = smape(y_test_reshaped, predictions_original)
print(f"sMAPE: {smape_value}%")


sMAPE: 197.29436202318874%


The score is way better than from where we started indicating that the likely cause of the poor result in previous codes was due to the small scale of the data that wee trained on.
Most liekly, if we use more data to train the LSTM(with null) model then it would perform better than this.

As we have noticed the lack of computational power and less sub sampling model, we will try a different approach to optimise in the next model/
We will use an advanced model but with focusing only on two assets and see if we can perform better. 